In [1]:
!pip install tensorflow numpy pandas matplotlib


In [3]:
import zipfile
import os
zip_path = '/content/Root Canal.v1i.yolov11.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/')
os.listdir('/content/')


['.config',
 'data.yaml',
 'Root Canal.v1i.yolov11.zip',
 'README.dataset.txt',
 'train',
 'test',
 'README.roboflow.txt',
 'valid',
 'sample_data']

In [12]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import classification_report, accuracy_score

label_mapping = {'1': 0, '2': 0, '3': 1}  # 0 = Root Canal, 1 = Filling

def load_images_and_labels(image_folder, label_folder, image_size=(128, 128)):
    images = []
    labels = []

    for filename in os.listdir(image_folder):
        if filename.endswith('.jpg'):
            image_path = os.path.join(image_folder, filename)
            label_path = os.path.join(label_folder, filename.replace('.jpg', '.txt'))

            image = cv2.imread(image_path)
            image = cv2.resize(image, image_size)
            image = image / 255.0

            # Load label (only the first number, ignore the rest)
            with open(label_path, 'r') as f:
                line = f.readline().strip()
                if line:
                    class_label = line.split()[0]

                    if class_label in label_mapping:  # Ensure label is valid
                        images.append(image)
                        labels.append(label_mapping[class_label])

    return np.array(images), np.array(labels)



train_images, train_labels = load_images_and_labels('/content/data/train/images', '/content/data/train/labels')
valid_images, valid_labels = load_images_and_labels('/content/data/valid/images', '/content/data/valid/labels')
test_images, test_labels = load_images_and_labels('/content/data/test/images', '/content/data/test/labels')


model = Sequential([
    Flatten(input_shape=(128, 128, 3)),
     Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
     Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


history = model.fit(
    train_images, train_labels,
    epochs=10,
    validation_data=(valid_images, valid_labels),
    batch_size=32
)



y_pred = (model.predict(test_images) > 0.5).astype("int32").flatten()


print("Accuracy:", accuracy_score(test_labels, y_pred))
print(classification_report(test_labels, y_pred, target_names=["Root Canal", "Filling"]))






/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten_5 (Flatten)                  │ (None, 49152)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 512)                 │      25,166,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,338,881 (96.66 MB)

 Trainable params: 25,338,881 (96.66 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 577ms/step - accuracy: 0.8355 - loss: 3.9142 - val_accuracy: 0.8929 - val_loss: 3.6661
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 569ms/step - accuracy: 0.8120 - loss: 3.6522 - val_accuracy: 0.8929 - val_loss: 3.0886
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 486ms/step - accuracy: 0.8258 - loss: 4.1730 - val_accuracy: 0.8929 - val_loss: 1.9880
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 496ms/step - accuracy: 0.8197 - loss: 2.2533 - val_accuracy: 0.8750 - val_loss: 0.6731
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 682ms/step - accuracy: 0.8057 - loss: 1.5521 - val_accuracy: 0.8929 - val_loss: 0.9945
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 487ms/step - accuracy: 0.8797 - loss: 1.3587 - val_accuracy: 0.6429 - val_loss: 1.5675
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 487ms/step - accuracy: 0.7328 - loss: 1.3704 - val_accuracy: 0.8929 - val_loss: 0.8002
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 534ms/step - accuracy: 0.8533 - loss: 1.0887 - val_accuracy: 0.8929 - val_loss:

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
